# Week 1 - Exercice 2

https://python.langchain.com/docs/how_to/output_parser_retry/

1. Explorez la documentation de OutputFixingParser et/ou RetryWithError.
2. Intégrez l'un de ces parsers correcteurs (ou les deux pour expérimenter) dans au moins l'une de vos chaînes de parsing existantes (par exemple, la chaîne short_chain ou long_chain).
3. Testez la chaîne modifiée pour voir comment elle gère les cas où le modèle fait une petite erreur de formatage initiale.

In [1]:
from langchain_ollama import ChatOllama, OllamaLLM
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers.fix import OutputFixingParser
from langchain.output_parsers.retry import RetryWithErrorOutputParser
from pprint import pprint

In [2]:
import sys

sys.path.append("../")

from agents.technical_analyst.technical_analysis_pydantic_model import TickerTechnicalAnalysis, ShortTimeframeData, LongTimeframeData

In [3]:
MODEL = "cogito:8b"
DEEP_THINKING_INSTRUCTION = "Enable deep thinking subroutine.\n\n"
invocation = {"stock": "APPLE"}

In [4]:
model = ChatOllama(model=MODEL, num_gpu=256, num_ctx=4092 * 2, num_predict=1000 * 2)
error_model = ChatOllama(model=MODEL, num_gpu=256, num_ctx=4092 * 4, num_predict=1000 * 2)

In [5]:
system_template = ("As trading technical analyst expert, your task is to generate the technical analysis report for {stock} at the specified JSON format. "
                   "Use the data (simulated for the exercice) to generate the technical analysis of {stock} action totally filling the JSON schema described below. "

                   "The format of your response is CRITICAL and MUST ADHERE EXACTLY to the JSON schema described here:"
                   "\n{format_instructions}\n"
                   "Once again, the JSON schema described above is CRITICAL and MUST BE RESPECTED."
                )

In [6]:
short_technical_parser = PydanticOutputParser(pydantic_object=ShortTimeframeData)


In [7]:

short_prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=system_template),
    HumanMessagePromptTemplate.from_template(template="{stock}"),
    ],
    input_variables=["stock"],
    partial_variables={"format_instructions": short_technical_parser.get_format_instructions(),
                    #    "output_json_schema": technical_parser._get_schema(pydantic_object=TickerTechnicalAnalysis)
                       }
)

In [8]:
retry_parser = RetryWithErrorOutputParser.from_llm(
    llm=error_model,
    # OllamaLLM(
    #     model=MODEL, num_gpu=256,
    #     #  num_ctx=4092 * 2, 
    #     #  num_predict=1000 * 1, 
    #             #   temperature=0
    #               ),
    parser=short_technical_parser,
    max_retries=1,
)

In [9]:
# from langchain_core.runnables import RunnableLambda, RunnableParallel

# completion_chain = short_prompt_template | model
# main_chain = RunnableParallel(
#     completion=completion_chain, prompt_value=short_prompt_template
# ) | RunnableLambda(lambda x: retry_parser.parse_with_prompt(**x))



In [10]:
test_1 = short_prompt_template.invoke({"stock": "APPLE"})
test_1

ChatPromptValue(messages=[SystemMessage(content='As trading technical analyst expert, your task is to generate the technical analysis report for APPLE at the specified JSON format. Use the data (simulated for the exercice) to generate the technical analysis of APPLE action totally filling the JSON schema described below. The format of your response is CRITICAL and MUST ADHERE EXACTLY to the JSON schema described here:\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"BollignerBandsRawValues": {"description": "Bollinger Bands raw values provided by the Bollinger Bands tool.

In [11]:
test_2 = model.invoke(test_1)
test_2

AIMessage(content='```json\n{\n    "data_timeframe": 5,\n    "supports_evaluation": {\n        "supports_evaluation": "Strong support zone at $123 identified",\n        "support_interaction_status": "PRICE_ABOVE_SUPPORT",\n        "support_interaction_implication": "POTENTIAL_BUY_ZONE",\n        "close_support_level": "$125.20 - Multiple confluence zones",\n        "middle_support_level": "$124.50 - Key historical level",\n        "far_support_level": "$123.00 - Major psychological support"\n    },\n    "resistances_evaluation": {\n        "resistances_evaluation": "Multiple resistance zones identified at $126-$127 range",\n        "resistance_interaction_status": "TESTING_RESISTANCE",\n        "resistance_interaction_implication": "TAKE_PROFIT_ZONE",\n        "close_resistance_level": "$125.90 - Immediate ceiling level",\n        "middle_resistance_level": "$126.50 - Key Fibonacci resistance",\n        "far_resistance_level": "$127.00 - Major horizontal resistance"\n    },\n    "price

In [12]:
import json
import ast

In [12]:
short_technical_parser.invoke(test_2)

OutputParserException: Failed to parse ShortTimeframeData from completion {"data_timeframe": 5, "supports_evaluation": {"supports_evaluation": "Strong support zone at $123 identified", "support_interaction_status": "PRICE_ABOVE_SUPPORT", "support_interaction_implication": "POTENTIAL_BUY_ZONE", "close_support_level": "$125.20 - Multiple confluence zones", "middle_support_level": "$124.50 - Key historical level", "far_support_level": "$123.00 - Major psychological support"}, "resistances_evaluation": {"resistances_evaluation": "Multiple resistance zones identified at $126-$127 range", "resistance_interaction_status": "TESTING_RESISTANCE", "resistance_interaction_implication": "TAKE_PROFIT_ZONE", "close_resistance_level": "$125.90 - Immediate ceiling level", "middle_resistance_level": "$126.50 - Key Fibonacci resistance", "far_resistance_level": "$127.00 - Major horizontal resistance"}, "prices_evaluation": {"prices_trend": "STRONG_BULLISH", "prices_trading_action": "BUY", "primary_prices

In [ ]:
# test_2_content = json.dumps(ast.literal_eval(test_2.content.replace("null", "None")))
# test_2_content

In [13]:
test_3 = retry_parser.parse_with_prompt(completion=test_2.content, prompt_value=short_prompt_template.format_prompt(stock="AAPL"))

OutputParserException: Failed to parse ShortTimeframeData from completion {"data_timeframe": 5, "supports_evaluation": {"supports_evaluation": "Strong support zone at $123 identified", "support_interaction_status": "PRICE_ABOVE_SUPPORT", "support_interaction_implication": "POTENTIAL_BUY_ZONE", "close_support_level": "$125.20 - Multiple confluence zones", "middle_support_level": "$124.50 - Key historical level", "far_support_level": "$123.00 - Major psychological support"}, "resistances_evaluation": {"resistances_evaluation": "Multiple resistance zones identified at $126-$127 range", "resistance_interaction_status": "TESTING_RESISTANCE", "resistance_interaction_implication": "TAKE_PROFIT_ZONE", "close_resistance_level": "$125.90 - Immediate ceiling level", "middle_resistance_level": "$126.50 - Key Fibonacci resistance", "far_resistance_level": "$127.00 - Major horizontal resistance"}, "prices_evaluation": {"prices_trend": "STRONG_BULLISH", "prices_trading_action": "BUY", "primary_prices

In [ ]:
test_3

ShortTimeframeData(data_timeframe=5, supports_evaluation=SupportEvaluation(evaluation='Strong Support Levels at $130 and $135', interaction_status=<SupportResistanceInteractionStatus.PRICE_ABOVE_SUPPORT: 'PRICE_ABOVE_SUPPORT'>, interaction_implication=<SupportResistanceInteractionImplication.TAKE_PROFIT_ZONE: 'TAKE_PROFIT_ZONE'>, close_level='$131.50', middle_level='$132.25', far_level='$133.00', raw_tool_data=SupportRawToolData(close_value=1315.0, middle_value=1322.5, far_value=1330.0)), resistances_evaluation=ResistanceEvaluation(evaluation='Strong Resistance Levels at $141 and $145', interaction_status=<SupportResistanceInteractionStatus.PRICE_BELOW_RESISTANCE: 'PRICE_BELOW_RESISTANCE'>, interaction_implication=<SupportResistanceInteractionImplication.POTENTIAL_BUY_ZONE: 'POTENTIAL_BUY_ZONE'>, close_level='$140.75', middle_level='$141.50', far_level='$142.25', raw_tool_data=ResistanceRawToolData(close_value=1407.5, middle_value=1415.0, far_value=1422.5)), prices_evaluation=PricesEva

In [ ]:
tmp

In [13]:
tmp.model_dump_json()

'{"data_timeframe":5,"supports_evaluation":{"evaluation":"Strong support levels identified at $130 and $135","interaction_status":"PRICE_ABOVE_SUPPORT","interaction_implication":"TAKE_PROFIT_ZONE","close_level":"$130","middle_level":"$132.5","far_level":"$135","raw_tool_data":{"close_value":130.0,"middle_value":132.5,"far_value":135.0}},"resistances_evaluation":{"evaluation":"Key resistance levels at $165 and $170","interaction_status":"PRICE_BELOW_RESISTANCE","interaction_implication":"STOP_LOSS_ZONE","close_level":"$165","middle_level":"$167.5","far_level":"$170","raw_tool_data":{"close_value":165.0,"middle_value":167.5,"far_value":170.0}},"prices_evaluation":{"trend":"BULLISH","trading_action":"BUY","primary_trend":"STRONG_BULLISH","primary_trend_number_of_touches":3,"secondary_trend":"BULLISH","secondary_trend_number_of_touches":2,"minor_trend":"CONSOLIDATION","minor_trend_number_of_touches":1,"raw_tool_data":{"prices_value":155.0},"trend_evaluation":"BULLISH","chart_pattern":"Doub

In [ ]:

short_chain = short_prompt_template | model | short_technical_parser
short_json = short_chain.invoke(invocation)
short_json

In [ ]:
pprint(short_json.model_dump_json())

('{"data_timeframe":5,"supports_evaluation":{"evaluation":"Strong support '
 'levels identified at $130 and '
 '$135","interaction_status":"PRICE_ABOVE_SUPPORT","interaction_implication":"TAKE_PROFIT_ZONE","close_level":"$130","middle_level":"$132.5","far_level":"$135","raw_tool_data":{"close_value":130.0,"middle_value":132.5,"far_value":135.0}},"resistances_evaluation":{"evaluation":"Key '
 'resistance levels at $165 and '
 '$170","interaction_status":"PRICE_BELOW_RESISTANCE","interaction_implication":"STOP_LOSS_ZONE","close_level":"$165","middle_level":"$167.5","far_level":"$170","raw_tool_data":{"close_value":165.0,"middle_value":167.5,"far_value":170.0}},"prices_evaluation":{"trend":"BULLISH","trading_action":"BUY","primary_trend":"STRONG_BULLISH","primary_trend_number_of_touches":3,"secondary_trend":"BULLISH","secondary_trend_number_of_touches":2,"minor_trend":"CONSOLIDATION","minor_trend_number_of_touches":1,"raw_tool_data":{"prices_value":155.0},"trend_evaluation":"BULLISH","cha

In [ ]:
long_technical_parser = PydanticOutputParser(pydantic_object=LongTimeframeData)

long_prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=system_template),
    HumanMessagePromptTemplate.from_template(template="{stock}"),
    ],
    input_variables=["stock"],
    partial_variables={"format_instructions": long_technical_parser.get_format_instructions(),
                    #    "output_json_schema": technical_parser._get_schema(pydantic_object=TickerTechnicalAnalysis)
                       }
)

long_chain = long_prompt_template | model | long_technical_parser
long_json = long_chain.invoke(invocation)
long_json

LongTimeframeData(data_timeframe=60, supports_evaluation=SupportEvaluation(evaluation='Strong Support Levels Present', interaction_status=<SupportResistanceInteractionStatus.PRICE_ABOVE_SUPPORT: 'PRICE_ABOVE_SUPPORT'>, interaction_implication=<SupportResistanceInteractionImplication.POTENTIAL_BUY_ZONE: 'POTENTIAL_BUY_ZONE'>, close_level='144.50', middle_level='139.25', far_level='134.00', raw_tool_data=SupportRawToolData(close_value=144.5, middle_value=139.25, far_value=134.0)), resistances_evaluation=ResistanceEvaluation(evaluation='Multiple Resistance Levels Active', interaction_status=<SupportResistanceInteractionStatus.PRICE_BELOW_RESISTANCE: 'PRICE_BELOW_RESISTANCE'>, interaction_implication=<SupportResistanceInteractionImplication.STOP_LOSS_ZONE: 'STOP_LOSS_ZONE'>, close_level='155.75', middle_level='150.50', far_level='145.25', raw_tool_data=ResistanceRawToolData(close_value=155.75, middle_value=150.5, far_value=145.25)), prices_evaluation=PricesEvaluation(trend='BULLISH', tradi

In [ ]:
pprint(long_json.model_dump_json())

('{"data_timeframe":60,"supports_evaluation":{"evaluation":"Strong Support '
 'Levels '
 'Present","interaction_status":"PRICE_ABOVE_SUPPORT","interaction_implication":"POTENTIAL_BUY_ZONE","close_level":"144.50","middle_level":"139.25","far_level":"134.00","raw_tool_data":{"close_value":144.5,"middle_value":139.25,"far_value":134.0}},"resistances_evaluation":{"evaluation":"Multiple '
 'Resistance Levels '
 'Active","interaction_status":"PRICE_BELOW_RESISTANCE","interaction_implication":"STOP_LOSS_ZONE","close_level":"155.75","middle_level":"150.50","far_level":"145.25","raw_tool_data":{"close_value":155.75,"middle_value":150.5,"far_value":145.25}},"prices_evaluation":{"trend":"BULLISH","trading_action":"OUTPERFORM","primary_trend":"STRONG_BULLISH","primary_trend_number_of_touches":5,"secondary_trend":"BEARISH","secondary_trend_number_of_touches":2,"minor_trend":"CONSOLIDATION","minor_trend_number_of_touches":1,"raw_tool_data":{"prices_value":149.5},"trend_evaluation":"Positive '
 'Mome

In [ ]:
global_system_template = ("As trading technical analyst expert, your task is to generate the technical analysis report for {stock} at the specified JSON format. "
                   "The JSON of the short timeframe analysis is {short_json}."
                   "The JSON of the long timeframe analysis is {long_json}."
                   "Use the data (simulated for the exercice) to generate the technical analysis of {stock} action totally filling the JSON schema described below. "

                   "The format of your response is CRITICAL and MUST ADHERE EXACTLY to the JSON schema described here:"
                   "\n{format_instructions}\n"
                   "Once again, the JSON schema described above is CRITICAL and MUST BE RESPECTED."
                )

In [ ]:
global_invocation = {"stock": "APPLE", "short_json":short_json.model_dump_json(), "long_json":long_json.model_dump_json()}

In [ ]:
global_technical_parser = PydanticOutputParser(pydantic_object=TickerTechnicalAnalysis)

global_prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=global_system_template),
    HumanMessagePromptTemplate.from_template(template="{stock}"),
    ],
    input_variables=["stock", "short_json", "long_json"],
    partial_variables={"format_instructions": global_technical_parser.get_format_instructions(),
                    #    "output_json_schema": technical_parser._get_schema(pydantic_object=TickerTechnicalAnalysis)
                       }
)

global_chain = global_prompt_template | model | global_technical_parser
global_json = global_chain.invoke(global_invocation)
global_json

TickerTechnicalAnalysis(name_of_the_company='Apple Inc.', isin_of_the_company='US0378331005', time_of_the_report=datetime.datetime(2023, 8, 16, 10, 0, tzinfo=TzInfo(UTC)), synthesis=Synthesis(conclusion='Strong bullish trend with multiple support levels at $130-$135 and resistance zones at $165-$170. Key technical patterns include a Double Bottom Pattern forming on the short timeframe, while a Bullish Flag Pattern is developing on the long timeframe.', synthese_remarkable_values=None, synthese_trading_action=<TradingActions.BUY: 'BUY'>, synthese_support_resistance_comment='Key support levels at $130-$135 and resistance zones at $165-$170. Multiple touches confirmed on support levels with strong volume confirmation.', synthese_support_resistance_interaction_status=<SupportResistanceInteractionStatus.PRICE_ABOVE_SUPPORT: 'PRICE_ABOVE_SUPPORT'>, synthese_support_resistance_interaction_implication=<SupportResistanceInteractionImplication.TAKE_PROFIT_ZONE: 'TAKE_PROFIT_ZONE'>))

In [ ]:
pprint(global_json.model_dump_json())

('{"name_of_the_company":"Apple '
 'Inc.","isin_of_the_company":"US0378331005","time_of_the_report":"2023-08-16T10:00:00Z","synthesis":{"conclusion":"Strong '
 'bullish trend with multiple support levels at $130-$135 and resistance zones '
 'at $165-$170. Key technical patterns include a Double Bottom Pattern forming '
 'on the short timeframe, while a Bullish Flag Pattern is developing on the '
 'long '
 'timeframe.","synthese_remarkable_values":null,"synthese_trading_action":"BUY","synthese_support_resistance_comment":"Key '
 'support levels at $130-$135 and resistance zones at $165-$170. Multiple '
 'touches confirmed on support levels with strong volume '
 'confirmation.","synthese_support_resistance_interaction_status":"PRICE_ABOVE_SUPPORT","synthese_support_resistance_interaction_implication":"TAKE_PROFIT_ZONE"}}')
